In [ ]:
import simpy
import random
import numpy as np
import pandas as pd
from datetime import timedelta

# Configuração da simulação
NUM_TERMINALS = 20
SHIP_ARRIVAL_RATE = 5  # navios por dia
SIM_DURATION = 30  # dias de simulação

# Distribuição de destino dos navios (exemplo: 50% dos navios vão para os 5 primeiros terminais)
DESTINATION_PROBS = [0.08] * 5 + [0.06] * 5 + [0.04] * 5 + [0.02] * 5

# Parâmetros do tempo de serviço por terminal (distribuição triangular)
SERVICE_TIMES = {i: (4, 8, 6) for i in range(NUM_TERMINALS)}  # (min, max, modo)

# Listas para armazenar resultados
ship_log = []
terminal_log = []

class Porto:
    def __init__(self, env):
        self.env = env
        self.terminals = {i: simpy.Resource(env, capacity=1) for i in range(NUM_TERMINALS)}
        self.terminal_status = {i: {'start': None, 'idle_time': 0} for i in range(NUM_TERMINALS)}

    def service(self, ship_id, arrival_time, terminal_id, duration_params):
        with self.terminals[terminal_id].request() as req:
            yield req  # Aguarda o terminal ficar disponível

            # Registra início do uso do terminal
            if self.terminal_status[terminal_id]['start'] is None:
                self.terminal_status[terminal_id]['start'] = self.env.now
            else:
                idle_time = self.env.now - self.terminal_status[terminal_id]['start']
                self.terminal_status[terminal_id]['idle_time'] += idle_time

            start_time = self.env.now
            print(f'Navio {ship_id} começou a ser atendido no terminal {terminal_id} em {start_time:.2f} horas')

            # Gera duração do serviço
            duration = random.triangular(*duration_params)
            yield self.env.timeout(duration)

            end_time = self.env.now
            print(f'Navio {ship_id} terminou em {end_time:.2f} horas | Tempo de fila: {(start_time - arrival_time):.2f}, Tempo de serviço: {duration:.2f}')

            # Armazena resultados
            ship_log.append({
                'ID Navio': ship_id,
                'Terminal': terminal_id,
                'Chegada': arrival_time,
                'Início Serviço': start_time,
                'Fila (h)': start_time - arrival_time,
                'Duração Serviço': duration,
                'Término': end_time
            })

            # Atualiza status do terminal
            self.terminal_status[terminal_id]['start'] = None

def ship_generator(env, porto):
    ship_id = 0
    while True:
        # Intervalo entre chegadas (Poisson)
        interarrival_time = random.expovariate(SHIP_ARRIVAL_RATE / 24)  # convertendo para horas
        yield env.timeout(interarrival_time)

        ship_id += 1
        destination = np.random.choice(range(NUM_TERMINALS), p=DESTINATION_PROBS)
        duration_params = SERVICE_TIMES[destination]

        arrival_time = env.now
        print(f'Navio {ship_id} chegou no porto em {arrival_time:.2f} horas')
    
        env.process(porto.service(ship_id, arrival_time, destination, duration_params))

def run_simulation():
    env = simpy.Environment()
    porto = Porto(env)
    env.process(ship_generator(env, porto))
    env.run(until=SIM_DURATION * 24)  # Converte dias para horas

    # DataFrame com logs dos navios
    df_ships = pd.DataFrame(ship_log)
    df_terminals = []

    for term_id, log in porto.terminal_status.items():
        df_terminals.append({
            'Terminal': term_id,
            'Tempo Ocioso (h)': log['idle_time'],
            'Tempo Total (h)': SIM_DURATION * 24,
            'Ocupação (%)': ((SIM_DURATION * 24 - log['idle_time']) / (SIM_DURATION * 24)) * 100
        })

    df_terminals = pd.DataFrame(df_terminals)

    return df_ships, df_terminals

# Roda a simulação
df_ships, df_terminals = run_simulation()

# Mostra resultados
print("\nLogs dos navios:")
display(df_ships.head(10).style.format(precision=2))

print("\nLogs dos terminais:")
display(df_terminals.style.format(precision=2))

Navio 1 chegou no porto em 2.94 horas
Navio 1 começou a ser atendido no terminal 13 em 2.94 horas
Navio 1 terminou em 8.04 horas | Tempo de fila: 0.00, Tempo de serviço: 5.10
Navio 2 chegou no porto em 14.61 horas
Navio 2 começou a ser atendido no terminal 10 em 14.61 horas
Navio 3 chegou no porto em 15.25 horas
Navio 3 começou a ser atendido no terminal 6 em 15.25 horas
Navio 3 terminou em 19.54 horas | Tempo de fila: 0.00, Tempo de serviço: 4.29
Navio 2 terminou em 20.04 horas | Tempo de fila: 0.00, Tempo de serviço: 5.43
Navio 4 chegou no porto em 40.97 horas
Navio 4 começou a ser atendido no terminal 12 em 40.97 horas
Navio 4 terminou em 46.21 horas | Tempo de fila: 0.00, Tempo de serviço: 5.23
Navio 5 chegou no porto em 48.46 horas
Navio 5 começou a ser atendido no terminal 5 em 48.46 horas
Navio 6 chegou no porto em 48.46 horas
Navio 6 começou a ser atendido no terminal 0 em 48.46 horas
Navio 7 chegou no porto em 52.56 horas
Navio 7 começou a ser atendido no terminal 4 em 52.56 h

,ID Navio,Terminal,Chegada,Início Serviço,Fila (h),Duração Serviço,Término
0,1,13,2.94,2.94,0.00,5.10,8.04
1,3,6,15.25,15.25,0.00,4.29,19.54
2,2,10,14.61,14.61,0.00,5.43,20.04
3,4,12,40.97,40.97,0.00,5.23,46.21
4,6,0,48.46,48.46,0.00,4.73,53.19
5,5,5,48.46,48.46,0.00,5.03,53.48
6,7,4,52.56,52.56,0.00,5.87,58.43
7,8,1,55.58,55.58,0.00,5.34,60.92
8,9,0,63.49,63.49,0.00,5.76,69.25
9,10,3,72.76,72.76,0.00,6.65,79.42



Logs dos terminais:


,Terminal,Tempo Ocioso (h),Tempo Total (h),Ocupação (%)
0,0,0,720,100.00
1,1,0,720,100.00
2,2,0,720,100.00
3,3,0,720,100.00
4,4,0,720,100.00
5,5,0,720,100.00
6,6,0,720,100.00
7,7,0,720,100.00
8,8,0,720,100.00
9,9,0,720,100.00
